In [1]:
import numpy as np
import pandas as pd
from equilibrator_api import ComponentContribution, Q_
CC = ComponentContribution()
from equilibrator_assets.generate_compound import create_compound, get_or_create_compound

In [2]:
from ast import literal_eval

In [3]:
def CacheGen(filepath):
    df = pd.read_csv(filepath, sep='\t') 
    
    Compounds = []
    Reagents = []
    Products = []
    
    for i in range(len(df)):
        if df['TYPE'][i] == -1:
            reagent = df['COMPOUND'][i]
            Reagents.append(reagent)
            if reagent not in Compounds:
                Compounds.append(reagent)
        else:
            product = df['COMPOUND'][i]
            Products.append(product)
            if product not in Compounds:
                Compounds.append(product)
    
    compound_cache = get_or_create_compound(CC.ccache, Compounds, mol_format="smiles", error_log='./ErrorLog.tsv')
    
    return(compound_cache, Compounds)

In [7]:
def ThermoGen(filepath, compound_cache, Compounds, name):
    df = pd.read_csv(filepath, sep='\t')
    IDs = []
    Rules = []
    Reagents = []
    Products = []
    for i in range(len(df)):
        if df['ID'][i] not in IDs:
            IDs.append(df['ID'][i])
            Rules.append(df['RULE'][i])
        
    mus = []
    sigma_vecs = []
    for c in compound_cache:
        mu = (CC.predictor.preprocess.get_compound_prediction(c))[0]
        sigma_vec = (CC.predictor.preprocess.get_compound_prediction(c))[1]
        mus.append(mu)
        sigma_vecs.append(sigma_vec)
    
    error_log = pd.read_csv('./ErrorLog.tsv', sep='\t')
    final_compounds = []
    for i in range(len(Compounds)):
        if error_log['status'][i] == 'valid':
            final_compounds.append(Compounds[i])

    print(len(mus))
    print(len(final_compounds))
    
    EnergyChanges = []
    i = 0
    while i < len(df):
        dummy_mus = []
        dummy_sigma_vecs = []
        dummy_compounds = []
        dummy_coefficients = []
        rxn_reactant = ""
        rxn_product = ""
        if df['TYPE'][i] == -1:
            reagent = df['COMPOUND'][i]
            dummy_compounds.append(reagent)
            dummy_coefficients.append(-1)
            rxn_reactant = reagent
        else:
            product = df['COMPOUND'][i]
            dummy_compounds.append(product)
            dummy_coefficients.append(1)
            rxn_product = product
        if i <= (len(df)-2):
            while (df['ID'][i] == df['ID'][i+1]):
                i+=1
                if df['TYPE'][i] == -1:
                    reagent = df['COMPOUND'][i]
                    dummy_compounds.append(reagent)
                    dummy_coefficients.append(-1)
                    if rxn_reactant != "":
                        rxn_reactant = rxn_reactant + ", " + reagent
                    else:
                        rxn_reactant = reagent
                else:
                    product = df['COMPOUND'][i]
                    dummy_compounds.append(product)
                    dummy_coefficients.append(1)
                    if rxn_product != "":
                        rxn_product = rxn_product + ", " + product
                    else:
                        rxn_product = product
                if i == (len(df)-1):
                    break
        i+=1
        rxn_reactant = [rxn_reactant]; rxn_product = [rxn_product]
        Reagents.append(rxn_reactant); Products.append(rxn_product)
        valid_reaction = True
        for m in range(len(dummy_compounds)):
            if dummy_compounds[m] not in final_compounds:
                valid_reaction = False
                break
            else: 
                dummy_mus.append(mus[final_compounds.index(dummy_compounds[m])])
                dummy_sigma_vecs.append(sigma_vecs[final_compounds.index(dummy_compounds[m])])
        if valid_reaction == True:
            S = np.zeros(len(dummy_compounds))
            for n in range(len(dummy_coefficients)):
                S[n] = dummy_coefficients[n]
            dummy_mus = Q_(dummy_mus, "kJ/mol")
            dummy_sigma_vecs = Q_(dummy_sigma_vecs, "kJ/mol")
            standard_dgs = S.T @ dummy_mus
            U = S.T @ dummy_sigma_vecs
            EnergyChanges.append(standard_dgs._magnitude.round(2))
        else:
            EnergyChanges.append('NaN')
    
    outputdata = {'ID':IDs, 'REAGENTS':Reagents, 'PRODUCTS':Products, 'RULE':Rules, 'ENERGY CHANGE':EnergyChanges}
    outputdf = pd.DataFrame(outputdata)
    outputdf.to_csv(f'{name}_WithThermo.tsv', header=None, index=None, sep='\t', mode='a')
    return(outputdf)
        

In [8]:
%%time
a, b = CacheGen('~/Desktop/Aditya/tsv_rels/rels_1.tsv')
a = ThermoGen('~/Desktop/Aditya/tsv_rels/rels_1.tsv', a, b, 'Rels_1')
a

21
21
CPU times: user 13.4 s, sys: 8.14 s, total: 21.5 s
Wall time: 35 s


,ID,REAGENTS,PRODUCTS,RULE,ENERGY CHANGE
0,5_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[C(CO)=O, C(C(C(CO)O)O)=O]",Retro Aldol,13.37
1,5_1,[C(C(C(C(C(CO)O)O)O)O)=O],"[C(CO)=O, C(C(C(CO)O)O)=O]",Knoevenagel H (inv),13.37
2,7_0,[C(C(C(C(C(CO)O)O)O)O)=O],[C(CO)(C(C(C(CO)O)O)O)=O],Keto-enol migration twice,-2.00
3,9_0,"[N, C(C(C(C(C(CO)O)O)O)O)=O]","[O, C(CN)(C(C(C(CO)O)O)O)=O]",Amadori/Heyns Rearrangement,-20.34
4,12_0,"[O, C(C(C(C(C(CO)O)O)O)O)=O, C(C(C(C(C(CO)O)O)...","[C(C(C(C(C(CO)O)O)O)O)(O)=O, C(C(C(C(C(CO)O)O)...","Cannizarro 2, glucose (oxidation)",-17.27
5,12_1,"[O, C(C(C(C(C(CO)O)O)O)O)=O, C(C(C(C(C(CO)O)O)...","[C(C(C(C(C(CO)O)O)O)O)(O)=O, C(C(C(C(C(CO)O)O)...","Cannizarro 2, glucose (reduction)",-17.27
6,14_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[O, C(C(CC(C(CO)O)O)=O)=O]",Elimination + enol to keto,-41.01
7,16_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[O, C(CC(C(C(CO)O)O)=O)=O]",Elimination + enol to keto,-53.84
8,18_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[O, C(C(C(CC(CO)O)=O)O)=O]",Elimination + enol to keto,-97.97
9,20_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[O, C(CC(C=O)O)(C(CO)O)=O]",Elimination + enol to keto,-41.01


In [9]:
%%time
a, b = CacheGen('~/Desktop/Aditya/tsv_rels/rels_2.tsv')
a = ThermoGen('~/Desktop/Aditya/tsv_rels/rels_2.tsv', a, b, 'Rels_2')
a

180
180
CPU times: user 1min 42s, sys: 1min 2s, total: 2min 45s
Wall time: 3min 1s


,ID,REAGENTS,PRODUCTS,RULE,ENERGY CHANGE
0,5_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[C(CO)=O, C(C(C(CO)O)O)=O]",Retro Aldol,13.37
1,5_1,[C(C(C(C(C(CO)O)O)O)O)=O],"[C(CO)=O, C(C(C(CO)O)O)=O]",Knoevenagel H (inv),13.37
2,7_0,[C(C(C(C(C(CO)O)O)O)O)=O],[C(CO)(C(C(C(CO)O)O)O)=O],Keto-enol migration twice,-2.00
3,9_0,"[N, C(C(C(C(C(CO)O)O)O)O)=O]","[O, C(CN)(C(C(C(CO)O)O)O)=O]",Amadori/Heyns Rearrangement,-20.34
4,12_0,"[O, C(C(C(C(C(CO)O)O)O)O)=O, C(C(C(C(C(CO)O)O)...","[C(C(C(C(C(CO)O)O)O)O)(O)=O, C(C(C(C(C(CO)O)O)...","Cannizarro 2, glucose (oxidation)",-17.27
...,...,...,...,...,...
280,432_0,[C(C1C(C(C(C(O)O1)O)O)O)O],[C(C(C(C(C(CO)O)O)O)O)=O],"Hemiacetal Formation for 6 membered rings, inv...",4.15
281,433_0,[C1C(C(C(C(C(O)O1)O)O)O)O],[C(C(C(C(C(CO)O)O)O)O)=O],"Hemiacetal Formation for 7 membered rings, inv...",7.01
282,435_0,[C(C(C(C(C(CO)O)O)O)O)(O)=O],"[O, C1(C(C(C(C(CO)O)O1)O)O)=O]","Ring Closure 5 membered O, O",-20.13
283,437_0,[C(C(C(C(C(CO)O)O)O)O)(O)=O],"[O, C1(C(C(C(C(CO)O1)O)O)O)=O]","Ring Closure 6 membered O, O",-22.83


In [10]:
%%time
a, b = CacheGen('~/Desktop/Aditya/tsv_rels/rels_3.tsv')
a = ThermoGen('~/Desktop/Aditya/tsv_rels/rels_3.tsv', a, b, 'Rels_3')
a

1866
1866
CPU times: user 16min 1s, sys: 8min 51s, total: 24min 53s
Wall time: 3h 26min 32s


,ID,REAGENTS,PRODUCTS,RULE,ENERGY CHANGE
0,5_0,[C(C(C(C(C(CO)O)O)O)O)=O],"[C(CO)=O, C(C(C(CO)O)O)=O]",Retro Aldol,13.37
1,5_1,[C(C(C(C(C(CO)O)O)O)O)=O],"[C(CO)=O, C(C(C(CO)O)O)=O]",Knoevenagel H (inv),13.37
2,7_0,[C(C(C(C(C(CO)O)O)O)O)=O],[C(CO)(C(C(C(CO)O)O)O)=O],Keto-enol migration twice,-2.0
3,9_0,"[N, C(C(C(C(C(CO)O)O)O)O)=O]","[O, C(CN)(C(C(C(CO)O)O)O)=O]",Amadori/Heyns Rearrangement,-20.34
4,12_0,"[O, C(C(C(C(C(CO)O)O)O)O)=O, C(C(C(C(C(CO)O)O)...","[C(C(C(C(C(CO)O)O)O)O)(O)=O, C(C(C(C(C(CO)O)O)...","Cannizarro 2, glucose (oxidation)",-17.27
...,...,...,...,...,...
3948,4987_0,[C(C(CC(C(C=O)O)O)N)=O],"[C(C(CO)O)=O, C=C(C=O)N]","Michael Addition 0,2, (reverse)",71.38
3949,4989_0,[C(C(C(CC(CO)=O)N)O)=O],"[C(CO)=O, C(N)=CC(CO)=O]","Michael Addition 0,2, (reverse)",72.8
3950,4991_0,[C(C(CC(C(CO)O)N)=O)=O],"[C(CO)O, C(N)=CC(C=O)=O]","Michael Addition 0,2, (reverse)",69.77
3951,4993_0,[C(C(C(CC(CO)N)=O)O)=O],"[CO, C(C(C(C=CN)=O)O)=O]","Michael Addition 0,2, (reverse)",64.91


In [ ]:
%%time
a, b = CacheGen('~/Desktop/Aditya/tsv_rels/rels_4.tsv')
a = ThermoGen('~/Desktop/Aditya/tsv_rels/rels_4.tsv', a, b, 'Rels_4')
a